<a href="https://colab.research.google.com/github/prateek0489/100daysToDS/blob/master/FirstNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from functools import partial
import string
import io
import os
import re
pd.set_option('display.max_colwidth', 80)


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
stopwords = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()

In [0]:
training_data=pd.read_csv("train.csv")

In [0]:
training_data['tweet_contains_url'] = training_data['tweet'].apply(lambda x: len(re.findall("(?P<url>https?://[^\s]+)", x)))
training_data['tweet_contains_smiley'] =training_data['tweet'].apply(lambda x: len(re.findall("((:|;)\))+", x)))

In [7]:
print(training_data['tweet_contains_url'].corr(training_data['label']))
print(training_data['tweet_contains_smiley'].corr(training_data['label']))

-0.5113744449299077
-0.06959690708945807


In [58]:
training_data['tweet_contains_smiley'].value_counts()

0    7701
1     210
2       6
3       3
Name: tweet_contains_smiley, dtype: int64

In [62]:
positive_tweets_with_smiley=len(training_data[(training_data.tweet_contains_smiley)>0 & (training_data.label==0)])
all_tweets_with_smiley=len(training_data[training_data.tweet_contains_smiley>0])
print(positive_tweets_with_smiley/all_tweets_with_smiley)

1.0


In [0]:
def clean_text(text):
    text = re.sub("(?P<url>https?://[^\s]+)","",text)
    text = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [word for word in tokens if word not in stopwords and len(word)>3]
    return text

In [0]:
training_data.head()

In [0]:
training_data['body_text_nostop'] = training_data['tweet'].apply(lambda x: clean_text(x.lower()))

training_data.head()

In [31]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text = [wn.lemmatize(word) for word in tokenized_text]
    return text

training_data['body_text_lemmatized'] = training_data['body_text_nostop'].apply(lambda x: lemmatizing(x))

training_data['tidy_tweet']=training_data['body_text_lemmatized'].apply(lambda x:" ".join([word for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [35]:
training_data.head()

,id,label,tweet,tweet_contains_url,tweet_contains_smiley,body_text_nostop,body_text_lemmatized,tidy_tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1MfQV #android #apps #beautiful...,1,0,"[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers...","[fingerprint, pregnancy, test, android, apps, beautiful, cute, health, igers...",fingerprint pregnancy test android apps beautiful cute health igers iphoneon...
1,2,0,Finally a transparant silicon case ^^ Thanks to my uncle :) #yay #Sony #Xper...,1,1,"[finally, transparant, silicon, case, thanks, uncle, sony, xperia, sonyexper...","[finally, transparant, silicon, case, thanks, uncle, sony, xperia, sonyexper...",finally transparant silicon case thanks uncle sony xperia sonyexperias
2,3,0,We love this! Would you go? #talk #makememories #unplug #relax #iphone #smar...,1,0,"[love, would, talk, makememories, unplug, relax, iphone, smartphone, wifi, c...","[love, would, talk, makememories, unplug, relax, iphone, smartphone, wifi, c...",love would talk makememories unplug relax iphone smartphone wifi connect
3,4,0,I'm wired I know I'm George I was made that way ;) #iphone #cute #daventry #...,1,1,"[wired, know, george, made, iphone, cute, daventry, home]","[wired, know, george, made, iphone, cute, daventry, home]",wired know george made iphone cute daventry home
4,5,1,What amazing service! Apple won't even talk to me about a question I have un...,0,0,"[amazing, service, apple, wont, even, talk, question, unless, 1995, stupid, ...","[amazing, service, apple, wont, even, talk, question, unless, 1995, stupid, ...",amazing service apple wont even talk question unless 1995 stupid support


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test =train_test_split(training_data[['tidy_tweet','tweet_contains_smiley','tweet_contains_url']],
                                                 training_data['label'],test_size=.2)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, stop_words='english',
                                   analyzer=clean_text)

tfidf_vect_fit=tfidf_vectorizer.fit(X_train['tidy_tweet'])
tfidf_train=tfidf_vect_fit.transform(X_train['tidy_tweet'])
tfidf_test=tfidf_vect_fit.transform(X_test['tidy_tweet'])

X_train_vect=pd.concat([X_train[['tweet_contains_smiley','tweet_contains_url']].reset_index(drop=True),
                      pd.DataFrame(tfidf_train.toarray())],axis=1)
X_test_vect=pd.concat([X_test[['tweet_contains_smiley','tweet_contains_url']].reset_index(drop=True),
                      pd.DataFrame(tfidf_test.toarray())],axis=1)
X_train_vect.head()

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:520: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'stop_words' will not be used"


,tweet_contains_smiley,tweet_contains_url,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,4215,4216,4217,4218,4219,4220,4221,4222,4223,4224,4225,4226,4227,4228,4229,4230,4231,4232,4233,4234,4235,4236,4237,4238,4239,4240,4241,4242,4243,4244,4245,4246,4247,4248,4249,4250,4251,4252,4253,4254
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [48]:
rf=RandomForestClassifier(n_jobs=-1)
k_fold=KFold(n_splits=5)
cross_val_score(rf,X_train_vect,y_train,cv=k_fold,scoring='accuracy',n_jobs=-1)

array([0.88958991, 0.8792423 , 0.89265983, 0.87687451, 0.8792423 ])

In [0]:
from sklearn.ensemble import  GradientBoostingClassifier

In [0]:
from sklearn.metrics import precision_recall_fscore_support as score

In [0]:
def train_GB(est,max_depth, lr):
  gb=GradientBoostingClassifier(n_estimators=est,max_depth=max_depth,learning_rate= lr)
  gb_model=gb.fit(X_train_vect,y_train)
  y_pred=gb_model.predict(X_test_vect)
  precision,recall,fscore,train_support=score(y_test,y_pred,pos_label=1,average='binary')
  print('Est: {}/Depth: {} /LR: {} --- Precision: {} /Recall: {} /Accuracy: {}'.format(
        est,max_depth,lr, round(precision,3), round(recall, 3),
        round((y_pred==y_test).sum()/len(y_pred),3)))

In [57]:
for n_est in [50,100,150]:
  for max_depth in [3,7,11,15]:
    for lr in [0.01,0.1,1]:
      train_GB(n_est,max_depth,lr)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est: 50/Depth: 3 /LR: 0.01 --- Precision: 0.0 /Recall: 0.0 /Accuracy: 0.751
Est: 50/Depth: 3 /LR: 0.1 --- Precision: 0.709 /Recall: 0.785 /Accuracy: 0.866
Est: 50/Depth: 3 /LR: 1 --- Precision: 0.738 /Recall: 0.734 /Accuracy: 0.869
Est: 50/Depth: 7 /LR: 0.01 --- Precision: 1.0 /Recall: 0.023 /Accuracy: 0.756
Est: 50/Depth: 7 /LR: 0.1 --- Precision: 0.751 /Recall: 0.795 /Accuracy: 0.883
Est: 50/Depth: 7 /LR: 1 --- Precision: 0.691 /Recall: 0.724 /Accuracy: 0.85
Est: 50/Depth: 11 /LR: 0.01 --- Precision: 0.778 /Recall: 0.053 /Accuracy: 0.76
Est: 50/Depth: 11 /LR: 0.1 --- Precision: 0.727 /Recall: 0.775 /Accuracy: 0.871
Est: 50/Depth: 11 /LR: 1 --- Precision: 0.679 /Recall: 0.744 /Accuracy: 0.848
Est: 50/Depth: 15 /LR: 0.01 --- Precision: 0.765 /Recall: 0.099 /Accuracy: 0.768
Est: 50/Depth: 15 /LR: 0.1 --- Precision: 0.726 /Recall: 0.785 /Accuracy: 0.872
Est: 50/Depth: 15 /LR: 1 --- Precision: 0.676 /Recall: 0.759 /Accuracy: 0.849
Est: 100/Depth: 3 /LR: 0.01 --- Precision: 0.714 /Recall: 

In [0]:
X_train